In [127]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from dateutil import parser
from datetime import datetime, timedelta
import requests
import openpyxl
from pandas_market_calendars import get_calendar
from requests.adapters import HTTPAdapter
from requests.exceptions import RequestException, HTTPError, ConnectionError, Timeout

# Funções

In [128]:
def workday() -> datetime:
    return get_calendar('BMF').schedule(
        data_ini=(datetime.now() - timedelta(days=90)).strftime("%d/%m/%Y"),
        data_fim=datetime.now().strftime("%d/%m/%Y")
    ).loc[::-1, ("market_open")].iloc[0].date()

# Main

In [129]:
USER = Path(os.path.expanduser("~"))

In [130]:
excel_path = USER / r"C:\Users\GiordanoBrunoGava\OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx"
dados_cotacao__aj_consolidado_excel = pd.read_excel(excel_path)

In [131]:
dados_cotacao__aj_consolidado_excel

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,26/01/2024,"266,83","125,08","11,92",NaN,"4,74",128967,"107,5",8836,14668,...,"92,5205139426","8,92255827584","85,9310514269","81,681182285","8,53010861097","8,56633458444","10,811163791","105,838616826","113,446595362","23,3000975927"
1,29/01/2024,"270,5","124,75","12,13","237,606672465","4,68",128502,106,8791,14884,...,"92,3441783257","8,91317598533","84,9453882851","81,6718483595","8,52076566616","8,59466865693","11,1112134366","106,050486669","113,637261909","23,0067169975"
2,30/01/2024,"270,13","123,57","12,05","237,955786001","4,6",127401,"104,72",8728,14814,...,"92,4369865451","8,90379369482","84,6731575127","81,5785091042","8,54879450059","8,6135580386","10,9893182681","105,45339893","113,656328564","22,7795836335"
3,31/01/2024,"266,62","123,95","11,84","231,951033185","4,58",127752,"106,37",8755,14648,...,"92,4833906548","8,91317598533","85,6494333864","81,5038377","8,5581374454","8,59466865693","9,58283555458","105,328203114","113,379862071","23,2338503615"
4,01/02/2024,"267,45","124,76","11,9","227,602075996","4,55",128481,"106,35",8800,14784,...,"91,7873290092","8,91317598533","85,6869824585","81,910081472","8,60552469592","8,65567783994","9,90163830297","104,581332165","112,273763993","23,3474170435"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,17/07/2024,"341,3","125,82","15,12","210,51","5,89",129450,"103,25",9188,19547,...,98,"9,3","86,9","68,9","9,9","9,13","9,6","103,59","116,71","22,39"
119,18/07/2024,"344,9","124,02","15,32","212,36","5,95",127652,"100,3",9054,19761,...,"98,35","9,35","86,44","68,79","9,82","9,1","9,52","103,06","115,93","21,68"
120,19/07/2024,"343,76",124,"15,26","214,5","5,97",127616,"100,63",9003,19682,...,"98,16","9,41","87,3","68,9","9,92","9,13","9,58","104,15","116,51","21,69"
121,22/07/2024,"345,27","124,26","15,39",NaN,"6,03",127859,"101,8",9073,19788,...,"98,12","9,4","87,29","69,35","9,97","9,12","9,54","104,2","115,53","22,23"


In [132]:
dados_cotacao__aj_consolidado_excel['Data'] = pd.to_datetime(dados_cotacao__aj_consolidado_excel['Data'], format="%d/%m/%Y")

In [133]:
ultima_linha_com_data = dados_cotacao__aj_consolidado_excel['Data'].last_valid_index()

In [134]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_1=(datetime.now() - timedelta(days=180)).strftime("%d/%m/%Y")
data_fim_1=(datetime.now() - timedelta(days=120)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_1}%26data_fim%3D{data_fim_1}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_1 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [135]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_2=(datetime.now() - timedelta(days=119)).strftime("%d/%m/%Y")
data_fim_2=(datetime.now() - timedelta(days=60)).strftime("%d/%m/%Y")

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_2}%26data_fim%3D{data_fim_2}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_2 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [136]:
# CONSTRUIR O PAYLOAD COM AS DATAS DINÂMICAS
data_ini_3=(datetime.now() - timedelta(days=59)).strftime("%d/%m/%Y")
data_fim_3=datetime.now().strftime("%d/%m/%Y") 

url = "https://www.comdinheiro.com.br/Clientes/API/EndPoint001.php"
payload = (
    f"username=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F"
    f"%26data_ini%3D{data_ini_3}%26data_fim%3D{data_fim_3}"
    "%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000"
    "%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29"
    "%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1"
    "%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1"
    "%26relat_alias_automatico%3Dcmd_alias_01&format=json3"
)
        
headers = {"Content-Type": "application/x-www-form-urlencoded"}
querystring = {"code": "import_data"}

#SOLICITAÇÃO DA API
response_3 = requests.post(url, data=payload, headers=headers, params=querystring, timeout=1800)

In [137]:
data_1 = list()
for lin in (table := response_1.json()["tables"]["tab0"]):
    data_1.append(table[lin])

In [138]:
dados_cotacao__aj_novos_1 = pd.DataFrame(data_1)

In [139]:
data_2 = list()
for lin in (table := response_2.json()["tables"]["tab0"]):
    data_2.append(table[lin])

In [140]:
dados_cotacao__aj_novos_2 = pd.DataFrame(data_2)

dados_cotacao__aj_novos_2 = dados_cotacao__aj_novos_2.iloc[1:].reset_index(drop=True)

In [141]:
data_3 = list()
for lin in (table := response_3.json()["tables"]["tab0"]):
    data_3.append(table[lin])

In [142]:
dados_cotacao__aj_novos_3 = pd.DataFrame(data_3)

dados_cotacao__aj_novos_3 = dados_cotacao__aj_novos_3.iloc[1:].reset_index(drop=True)

In [143]:
dados_cotacao__aj_novos = pd.concat([dados_cotacao__aj_novos_1, dados_cotacao__aj_novos_2, dados_cotacao__aj_novos_3], ignore_index=True) 

In [144]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.rename(
    columns = {
        # Dict comprehension para colocar os nomes antigos da coluna com os valores da primeira linha.
        nome_antigo_da_coluna: nome_novo_da_coluna for
            nome_antigo_da_coluna, nome_novo_da_coluna in
                # Zip para poder fazer unpack das duas informacoes
                zip(
                    list(dados_cotacao__aj_novos.columns), [
                        # Como os dados da primeira linha vem em formatod indeseja ja formatamos aqui
                        str(item).replace("\nPRECO_AJ(,,,A,C)", "") for
                            item in
                                # iloc para pegar a primeira linha e todas as colunas
                                list(dados_cotacao__aj_novos.iloc[0, :])
                    ]
                )
        }
)

In [145]:
dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(index = 0)

In [146]:
# ARRUMA AS DATAS ERRADAS PARA O FORMATO DAS OUTRAS
parser.parse("12-06-2024", dayfirst = True)

datetime.datetime(2024, 6, 12, 0, 0)

In [147]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,26/01/2024,"266,83","125,08","11,92",,"4,74",128967,"107,5",8836,14668,...,"92,5205139426","8,92255827584","85,9310514269","81,681182285","8,53010861097","8,56633458444","10,811163791","105,838616826","113,446595362","23,3000975927"
2,29/01/2024,"270,5","124,75","12,13","237,606672465","4,68",128502,106,8791,14884,...,"92,3441783257","8,91317598533","84,9453882851","81,6718483595","8,52076566616","8,59466865693","11,1112134366","106,050486669","113,637261909","23,0067169975"
3,30/01/2024,"270,13","123,57","12,05","237,955786001","4,6",127401,"104,72",8728,14814,...,"92,4369865451","8,90379369482","84,6731575127","81,5785091042","8,54879450059","8,6135580386","10,9893182681","105,45339893","113,656328564","22,7795836335"
4,31/01/2024,"266,62","123,95","11,84","231,951033185","4,58",127752,"106,37",8755,14648,...,"92,4833906548","8,91317598533","85,6494333864","81,5038377","8,5581374454","8,59466865693","9,58283555458","105,328203114","113,379862071","23,2338503615"
5,01/02/2024,"267,45","124,76","11,9","227,602075996","4,55",128481,"106,35",8800,14784,...,"91,7873290092","8,91317598533","85,6869824585","81,910081472","8,60552469592","8,65567783994","9,90163830297","104,581332165","112,273763993","23,3474170435"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,18/07/2024,"344,9","124,02","15,32","212,36","5,95",127652,"100,3",9054,19761,...,"98,35","9,35","86,44","68,79","9,82","9,1","9,52","103,06","115,93","21,68"
122,19/07/2024,"343,76",124,"15,26","214,5","5,97",127616,"100,63",9003,19682,...,"98,16","9,41","87,3","68,9","9,92","9,13","9,58","104,15","116,51","21,69"
123,22/07/2024,"345,27","124,26","15,39",,"6,03",127859,"101,8",9073,19788,...,"98,12","9,4","87,29","69,35","9,97","9,12","9,54","104,2","115,53","22,23"
124,23/07/2024,"345,5","122,95","15,38","216,91","5,93",126589,"99,8",8927,19850,...,"97,83","9,36","86,14","70,07","9,91","9,13","9,53","104,15","115,5","21,91"


In [148]:
#dados_cotacao__aj_novos["Data"] = dados_cotacao__aj_novos["Data"].map(lambda row: parser.parse(row, dayfirst = True))

In [149]:
#dados_cotacao__aj_novos = dados_cotacao__aj_novos.rename(columns = {"Data": "DATA"}).astype(dtype = {"DATA": "datetime64[ns]"})

In [150]:
dados_cotacao__aj_novos.reset_index(drop = True)

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
0,26/01/2024,"266,83","125,08","11,92",,"4,74",128967,"107,5",8836,14668,...,"92,5205139426","8,92255827584","85,9310514269","81,681182285","8,53010861097","8,56633458444","10,811163791","105,838616826","113,446595362","23,3000975927"
1,29/01/2024,"270,5","124,75","12,13","237,606672465","4,68",128502,106,8791,14884,...,"92,3441783257","8,91317598533","84,9453882851","81,6718483595","8,52076566616","8,59466865693","11,1112134366","106,050486669","113,637261909","23,0067169975"
2,30/01/2024,"270,13","123,57","12,05","237,955786001","4,6",127401,"104,72",8728,14814,...,"92,4369865451","8,90379369482","84,6731575127","81,5785091042","8,54879450059","8,6135580386","10,9893182681","105,45339893","113,656328564","22,7795836335"
3,31/01/2024,"266,62","123,95","11,84","231,951033185","4,58",127752,"106,37",8755,14648,...,"92,4833906548","8,91317598533","85,6494333864","81,5038377","8,5581374454","8,59466865693","9,58283555458","105,328203114","113,379862071","23,2338503615"
4,01/02/2024,"267,45","124,76","11,9","227,602075996","4,55",128481,"106,35",8800,14784,...,"91,7873290092","8,91317598533","85,6869824585","81,910081472","8,60552469592","8,65567783994","9,90163830297","104,581332165","112,273763993","23,3474170435"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,18/07/2024,"344,9","124,02","15,32","212,36","5,95",127652,"100,3",9054,19761,...,"98,35","9,35","86,44","68,79","9,82","9,1","9,52","103,06","115,93","21,68"
121,19/07/2024,"343,76",124,"15,26","214,5","5,97",127616,"100,63",9003,19682,...,"98,16","9,41","87,3","68,9","9,92","9,13","9,58","104,15","116,51","21,69"
122,22/07/2024,"345,27","124,26","15,39",,"6,03",127859,"101,8",9073,19788,...,"98,12","9,4","87,29","69,35","9,97","9,12","9,54","104,2","115,53","22,23"
123,23/07/2024,"345,5","122,95","15,38","216,91","5,93",126589,"99,8",8927,19850,...,"97,83","9,36","86,14","70,07","9,91","9,13","9,53","104,15","115,5","21,91"


In [151]:
dados_cotacao__aj_novos

,Data,IVVB11,BOVA11,NASD11,USSX34,XINA11,IBOV,SMAL11,IDIV,BDRX,...,CPTI11,BODB11,BDIF11,XPID11,OGIN11,GARE11,MCHY11,XPLG11,LVBI11,ALOS3
1,26/01/2024,"266,83","125,08","11,92",,"4,74",128967,"107,5",8836,14668,...,"92,5205139426","8,92255827584","85,9310514269","81,681182285","8,53010861097","8,56633458444","10,811163791","105,838616826","113,446595362","23,3000975927"
2,29/01/2024,"270,5","124,75","12,13","237,606672465","4,68",128502,106,8791,14884,...,"92,3441783257","8,91317598533","84,9453882851","81,6718483595","8,52076566616","8,59466865693","11,1112134366","106,050486669","113,637261909","23,0067169975"
3,30/01/2024,"270,13","123,57","12,05","237,955786001","4,6",127401,"104,72",8728,14814,...,"92,4369865451","8,90379369482","84,6731575127","81,5785091042","8,54879450059","8,6135580386","10,9893182681","105,45339893","113,656328564","22,7795836335"
4,31/01/2024,"266,62","123,95","11,84","231,951033185","4,58",127752,"106,37",8755,14648,...,"92,4833906548","8,91317598533","85,6494333864","81,5038377","8,5581374454","8,59466865693","9,58283555458","105,328203114","113,379862071","23,2338503615"
5,01/02/2024,"267,45","124,76","11,9","227,602075996","4,55",128481,"106,35",8800,14784,...,"91,7873290092","8,91317598533","85,6869824585","81,910081472","8,60552469592","8,65567783994","9,90163830297","104,581332165","112,273763993","23,3474170435"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,18/07/2024,"344,9","124,02","15,32","212,36","5,95",127652,"100,3",9054,19761,...,"98,35","9,35","86,44","68,79","9,82","9,1","9,52","103,06","115,93","21,68"
122,19/07/2024,"343,76",124,"15,26","214,5","5,97",127616,"100,63",9003,19682,...,"98,16","9,41","87,3","68,9","9,92","9,13","9,58","104,15","116,51","21,69"
123,22/07/2024,"345,27","124,26","15,39",,"6,03",127859,"101,8",9073,19788,...,"98,12","9,4","87,29","69,35","9,97","9,12","9,54","104,2","115,53","22,23"
124,23/07/2024,"345,5","122,95","15,38","216,91","5,93",126589,"99,8",8927,19850,...,"97,83","9,36","86,14","70,07","9,91","9,13","9,53","104,15","115,5","21,91"


In [152]:
if dados_cotacao__aj_consolidado_excel.index.duplicated().any():
    dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.reset_index(drop=True)
if dados_cotacao__aj_novos.index.duplicated().any():
    dados_cotacao__aj_novos = dados_cotacao__aj_novos.reset_index(drop=True)

In [153]:
dados_cotacao__aj_consolidado_excel = dados_cotacao__aj_consolidado_excel.loc[:, ~dados_cotacao__aj_consolidado_excel.columns.duplicated()]
dados_cotacao__aj_novos = dados_cotacao__aj_novos.loc[:, ~dados_cotacao__aj_novos.columns.duplicated()]

In [154]:
# SALVA O DF CONCATENANDO NO EXCEL
output_path = USER / r'OneDrive - WISIR\General\3 - OPERACIONAL\1 - DADOS\01 - PYTHON\02 - CONSOLIDADOS\COTAJ_ACAO_CONSOLIDADO.xlsx'

dados_cotacao__aj_novos = dados_cotacao__aj_novos.drop(dados_cotacao__aj_novos.index[-1])
dados_cotacao__aj_novos.to_excel(output_path, index=False)

df = pd.read_excel(output_path)
df.replace(['-', '[]'], np.nan, inplace=True)
df = df.dropna(how='all', subset=df.columns.difference(['DATA']))
df.to_excel(output_path, index=False)


print('Dados atualizados e salvos no Excel.')
print('URL construída:', url + payload)

Dados atualizados e salvos no Excel.
URL construída: https://www.comdinheiro.com.br/Clientes/API/EndPoint001.phpusername=wisir.research&password=wisir.research&URL=HistoricoIndicadoresFundamentalistas001.php%3F%26data_ini%3D26/05/2024%26data_fim%3D24/07/2024%26trailing%3D12%26conv%3DMIXED%26moeda%3DMOEDA_ORIGINAL%26c_c%3Dconsolidado%26m_m%3D1000000%26n_c%3D2%26f_v%3D0%26papel%3Dexplode%28STOCK_GUIDE_WISIR%29%26indic%3DPRECO_AJ%28%2C%2C%2CA%2CC%29%26periodicidade%3Ddu%26graf_tab%3Dtabela%26desloc_data_analise%3D1%26flag_transpor%3D0%26c_d%3Dc%26enviar_email%3D0%26enviar_email_log%3D0%26cabecalho_excel%3Dmodo1%26relat_alias_automatico%3Dcmd_alias_01&format=json3
